In [19]:
# !pip install llama-index
# !pip install llama-index-embeddings-llamafile
# !pip install llama-index-llms-llamafil
# !pip install llama-index-readers-web


In [6]:
# chmod +x
# ./TinyLlama-1.1B-Chat-v1.0.F16.llamafile --server --nobrowser --embedding --port 8080
# ./Phi-3-mini-4k-instruct.F16.llamafile --server --nobrowser --embedding --port 8080
# ./mistral-7b-instruct-v0.2.Q4_K_M.llamafile --server --nobrowser --embedding --port 8080
# ./mistral-7b-instruct-v0.2.Q5_K_M.llamafile --server --nobrowser --embedding --port 8080

In [20]:
# Configure LlamaIndex
from llama_index.core import Settings
from llama_index.embeddings.llamafile import LlamafileEmbedding
from llama_index.llms.llamafile import Llamafile
from llama_index.core.node_parser import SentenceSplitter

Settings.embed_model = LlamafileEmbedding(base_url="http://localhost:8080")

Settings.llm = Llamafile(
	base_url="http://localhost:8080",
	temperature=0,
	seed=0
)

# Also set up a sentence splitter to ensure texts are broken into semantically-meaningful chunks (sentences) that don't take up the model's entire
# context window (2048 tokens). Since these chunks will be added to LLM prompts as part of the RAG process, we want to leave plenty of space for both
# the system prompt and the user's actual question.
Settings.transformations = [
	SentenceSplitter(
    	chunk_size=256,
    	chunk_overlap=5
	)
]

import pandas as pd
from llama_index.core import Document
from llama_index.core import VectorStoreIndex

# Load the Excel file
excel_path = '/Users/xzh/Desktop/AIPI561/Virtual-Fitness-Coach-Project/data/functionalDataBase_done.xlsx'
df = pd.read_excel(excel_path)

# Convert DataFrame to documents
docs = []
for index, row in df.iterrows():
    content = " ".join([f"{col}: {row[col]}" for col in df.columns])
    doc = Document(text=content)
    docs.append(doc)

# Build the index
index = VectorStoreIndex.from_documents(
    docs,
    show_progress=True,
)

In [21]:
# Save the index
index.storage_context.persist(persist_dir="./storage")

In [22]:
query_engine = index.as_query_engine()


In [23]:
question = "what are some of the variations of push-ups?"

In [24]:
print(query_engine.query(question))

ReadTimeout: timed out

### CalorieNinjas API

In [5]:
import requests
import os
from dotenv import load_dotenv


In [6]:
load_dotenv()
X_API_KEY = os.getenv('CALORIE_API_KEY')

In [7]:
api_url = 'https://api.calorieninjas.com/v1/nutrition?query='
query = '3lb carrots and a chicken sandwich'

In [12]:
def get_nutrition(query):
    response = requests.get(api_url + query, headers={'X-Api-Key': X_API_KEY})
    if response.status_code == requests.codes.ok:
        return response.json()
    else:
        return f"Error: {response.status_code} {response.text}"

In [13]:
print(get_nutrition("3lb carrots and a chicken sandwich"))

{'items': [{'name': 'carrots', 'calories': 477.8, 'serving_size_g': 1360.7759999999998, 'fat_total_g': 2.3, 'fat_saturated_g': 0.3, 'protein_g': 10.3, 'sodium_mg': 775, 'potassium_mg': 410, 'cholesterol_mg': 0, 'carbohydrates_total_g': 110.2, 'fiber_g': 40.7, 'sugar_g': 46.7}, {'name': 'chicken sandwich', 'calories': 243.9, 'serving_size_g': 100.0, 'fat_total_g': 11.2, 'fat_saturated_g': 2.1, 'protein_g': 16.2, 'sodium_mg': 767, 'potassium_mg': 184, 'cholesterol_mg': 35, 'carbohydrates_total_g': 20.9, 'fiber_g': 1.4, 'sugar_g': 3.6}]}


{
  "items": [
    {
      "name": "carrots",
      "calories": 477.8,
      "serving_size_g": 1360.776,
      "fat_total_g": 2.3,
      "fat_saturated_g": 0.3,
      "protein_g": 10.3,
      "sodium_mg": 775,
      "potassium_mg": 410,
      "cholesterol_mg": 0,
      "carbohydrates_total_g": 110.2,
      "fiber_g": 40.7,
      "sugar_g": 46.7
    },
    {
      "name": "chicken sandwich",
      "calories": 243.9,
      "serving_size_g": 100.0,
      "fat_total_g": 11.2,
      "fat_saturated_g": 2.1,
      "protein_g": 16.2,
      "sodium_mg": 767,
      "potassium_mg": 184,
      "cholesterol_mg": 35,
      "carbohydrates_total_g": 20.9,
      "fiber_g": 1.4,
      "sugar_g": 3.6
    }
  ]
}


In [23]:
!pip install langchain_community

  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 15.7 MB/s eta 0:00:00
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached langchain_text_splitters-0.2.2-py3-none-any.whl (25 kB)
Using cached jsonpointer-3.0.0-py2.py3-none-any.whl (7.6 kB)


In [12]:
from langchain_community.llms.llamafile import Llamafile

llm = Llamafile()

In [13]:
query = "Where to find a fitness coach?"

for chunks in llm.stream(query):
    print(chunks, end="")

print()



When you are looking for a fitness coach, there are several places where you can search. Here are some suggestions:

1. Gyms and Fitness Centers: Many gyms and fitness centers have personal trainers or group fitness instructors who offer coaching services. You can ask the front desk or receptionist for recommendations or simply approach a trainer and ask if they offer coaching services.
2. Online: There are many online fitness coaching platforms that connect clients with certified coaches. These platforms often offer customized plans, regular check-ins, and virtual sessions. Some popular options include MyFitnessPal, Fitbod, and TrainerRoad.
3. Local Community Centers: Your local community center may offer fitness classes or have personal trainers on staff. Check their website or call to inquire about coaching services.
4. Social Media: Many fitness coaches have a social media presence, so you can search for coaches on platforms like Instagram, Facebook, and Twitter. Look for coaches

In [1]:
from langchain_community.document_loaders import TextLoader, CSVLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import LlamafileEmbeddings
from langchain_chroma import Chroma
# from langchain_community.vectorstores.chroma import Chroma



### DB setup

In [2]:
loader = CSVLoader(file_path="data/functionalDataBase_done.csv")

documents = loader.load()

text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = LlamafileEmbeddings()
# vectorstore = Chroma.from_documents(texts[:3], embeddings, persist_directory="./chroma_DB_", collection_name='test_db__')
# vectorstore.persist()

In [4]:
vector_store = Chroma(
    collection_name="v_db",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

In [5]:
vector_store.add_documents(texts)

['e864eda0-a673-400b-998b-be7797aa3cb3',
 'b6df7ade-7808-47bb-9476-a1b9fad4807e',
 'b8742149-521e-4afb-a23f-ea11bbe23579',
 '6a3a16a7-f51e-4e18-9a0b-d3fa6dcd3df3',
 'daf05886-b5e9-4838-81b3-225ec188f1e1',
 '7a03b0c3-735c-4d0e-ac19-bb56886d2b86',
 'e2bb1cb0-819b-4a97-a493-e3b8ab33e5d0',
 '63bdfa73-7757-4c4f-b5d4-e139ce47450b',
 '3f4bd9d4-fb37-4af1-b665-0295912bb61e',
 'b474718e-9557-491f-9fc6-87fbf01a7ab4',
 '87317c66-437c-4470-b628-91c561cb3714',
 '39463f83-9d6b-40fa-8f37-0fe5eae9b681',
 '1c051c2c-e840-4756-82ef-f1b4674499f3',
 '88f22bf3-9895-49b7-9055-dc8305643af3',
 'a5452385-5352-43d4-849d-557e741612bb',
 '8d82c267-78dd-4c39-8fdc-3d9b9a46a44c',
 '926314f7-2d87-4040-92a8-17c4ec20f118',
 'c85733ec-e435-471a-b058-b42adc31e616',
 '8fe63a69-ed29-4911-876a-0fe2c65817c7',
 'd3505f95-7b28-493c-a9c2-d43fe0acfd7f',
 '6f7604d7-0761-4eab-a484-089761c0eae2',
 '40bbcaa2-a1f3-4e4c-a0a4-725cb4f28d37',
 '782a7f06-f2c4-48fb-8c09-200ac418eefa',
 '5643ed36-50c6-4ca6-a8c3-7a725b342baa',
 '4fb243c4-1bc8-

In [7]:
import chromadb

# Create a client
client = chromadb.PersistentClient("./chroma_langchain_db")

# Get all collections
collections = client.list_collections()

# Print the collections
for collection in collections:
    print(f"Collection name: {collection.name}")
    print(f"Collection metadata: {collection.metadata}")
    print("---")

Collection name: v_db
Collection metadata: None
---


#### Load DB from disk

In [132]:
# !pip install -U langchain-chroma

In [193]:
# !pip install --upgrade chromadb==0.4.14

In [3]:
embeddings = LlamafileEmbeddings()

In [9]:
import chromadb

persistent_client = chromadb.PersistentClient("./chroma_langchain_db")
# collection = persistent_client.get_or_create_collection("chroma_db")

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="v_db",
    embedding_function=embeddings,
)

In [12]:
results = vector_store_from_client.similarity_search(
    "how to do a push-up? ",
    k=2
)
for res in results:
    print(res.page_content)
    # print(f"* {res.page_content} [{res.metadata}]")

Exercise: Archer Push Up
Short YouTube Demonstration: https://youtu.be/_Qbz_54_9Sw
In-Depth YouTube Explanation: https://youtu.be/A0r8ploEnZY
Difficulty Level: Intermediate
Target Muscle Group: Chest
Prime Mover Muscle: Pectoralis Major
Secondary Muscle: 
Tertiary Muscle: 
Primary Equipment: Bodyweight
# Primary Items: 0
Secondary Equipment: 
# Secondary Items: 
Posture: Prone
Single or Double Arm: Double Arm
Continuous or Alternating Arms: Alternating
Grip: Flat Palm
Load Position (Ending): No Load
Combination Exercises: Single Exercise
Movement Pattern #1: Horizontal Push
Movement Pattern #2: 
Movement Pattern #3: 
Plane Of Motion #1: Frontal Plane
Plane Of Motion #2: 
Plane Of Motion #3: 
Body Region: Upper Body
Force Type: Push
Mechanics: Compound
Laterality: Unilateral
Exercise Classification: Calisthenics
﻿Exercise: Parallette Kneeling Push Up
Short YouTube Demonstration: https://youtu.be/iHR7BprMyEc
In-Depth YouTube Explanation: 
Difficulty Level: Novice
Target Muscle Group: Che

test


In [2]:
embeddings = LlamafileEmbeddings()


In [3]:
retriever = vector_store_from_client.as_retriever(search_type="similarity", search_kwargs={"k": 5})




NameError: name 'vector_store_from_client' is not defined

In [149]:
retriever.invoke("Youtube")



[]

In [143]:
len(docs)

0

In [23]:
import chromadb
from langchain_chroma import Chroma


persistent_client = chromadb.PersistentClient("./chroma_langchain_db")
# collection = persistent_client.get_or_create_collection("chroma_db")

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="v_db",
    embedding_function=embeddings,
)

retriever = vector_store_from_client.as_retriever(search_type="similarity", search_kwargs={"k": 5})


from langchain_community.llms.llamafile import Llamafile

llm = Llamafile()

from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
template = """Answer the question based on the following context, try your best to provide YouTube links if possible.:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "As a beginner, how to do push-ups?"
# result = chain.invoke(query)
# print(result)
for chunk in chain.stream(query):
    print(chunk, end="", flush=True)


Answer: For a beginner, I would recommend starting with the Parallette Kneeling Push Up or the Wide Push Up. Both exercises are classified as novice level and target the chest muscle group primarily. Here are some resources to help you get started:

1. Parallette Kneeling Push Up: This exercise involves using parallettes, which help provide support and stability. You can watch a short demonstration here: https://youtu.be/iHR7BprMyEc. For a more in-depth explanation, please refer to this video: https://youtu.be/UEZy-OmXdZs.
2. Wide Push Up: This exercise can be done on the floor with your hands placed wider than shoulder width apart. It also targets the chest muscle group and is demonstrated in a short video here: https://youtu.be/UEZy-OmXdZs. For more information, please refer to this video: https://youtu.be/Uvn1s5d0F6g.

Both exercises are considered compound movements, meaning they engage multiple muscle groups, and follow the horizontal push movement pattern in the sagittal plane o

In [17]:
# !pip install langchainhub
from langchain import hub

In [18]:
prompt = hub.pull("rlm/rag-prompt")


In [19]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)